# Import libraries

In [9]:
import os
import json
import random
import shutil

# Anonymizing Student Names

In [10]:
# Define the source and destination directories
source_directory = 'strategies'
anonymized_directory = 'Anonymized_Strategies'

# Dictionary to store the name mapping
name_mapping = {}

# Create the new directory if it doesn't exist
os.makedirs(anonymized_directory, exist_ok=True)
print(f"Created new directory: '{anonymized_directory}'")

Created new directory: 'Anonymized_Strategies'


In [11]:
# Get a list of all .ipynb files in the source directory
ipynb_files = [f for f in os.listdir(source_directory) if f.endswith('.ipynb')]

# Commented for not revealing the names
# ipynb_files

Using the anonymizer below, the true names will be altered to anonymized ones.

In [12]:
# LEN is the number of ipynb files
LEN = len(ipynb_files)

if not ipynb_files:
    print(f"No .ipynb files found in the '{source_directory}' directory.")
else:
    # A set to keep track of used numbers to ensure uniqueness
    used_numbers = set()

    for old_filename in ipynb_files:
        # Generate a unique random number between 1 and LEN
        while True:
            random_number = random.randint(1, LEN)
            if random_number not in used_numbers:
                used_numbers.add(random_number)
                break

        new_name = f'Student_{random_number:03d}'
        new_filename = f'{new_name}.ipynb'

        old_filepath = os.path.join(source_directory, old_filename)
        new_filepath = os.path.join(anonymized_directory, new_filename)
        
        # Copy the file to the new directory
        shutil.copy(old_filepath, new_filepath)
        # Commented print for not revealing the names
        # print(f"Copied '{old_filename}' to '{anonymized_directory}' as '{new_filename}'")
        
        # Modify the content of the new file
        with open(new_filepath, 'r+', encoding='utf-8') as f:
            notebook_data = json.load(f)

            for cell in notebook_data['cells']:
                if cell['cell_type'] == 'code':
                    source = cell['source']
                    for i, line in enumerate(source):
                        # Look for the 'def' keyword to find the function definition
                        if line.strip().startswith('def'):
                         # Create the new function definition line
                            new_line = f"def {new_name}(history):\n"
                            source[i] = new_line
                            # Commented print for not revealing the names
                            #print(f"Function name in '{new_filename}' changed to '{new_name}'")
                            break # Assume one function per notebook

                    # Replace the old source with the modified source
                    cell['source'] = source

            # Go back to the beginning of the file and write the new data
            f.seek(0)
            json.dump(notebook_data, f, indent=4)
            f.truncate()
            
            # Add the old and new name to the dictionary
            name_mapping[old_filename] = new_filename

    # Save the name mapping dictionary to a JSON file
    json_filepath = os.path.join('','name_mapping.json')
    with open(json_filepath, 'w', encoding='utf-8') as json_file:
        json.dump(name_mapping, json_file, indent=4)
        print(f"Saved name mapping to '{json_filepath}'")

Saved name mapping to 'name_mapping.json'
